In [1]:
import pandas as pd
import random
import wordninja

In [2]:
pos_locations = ['E:/qrf/Lab Stuff/newDataset/spring-boot_labeled_sentences.csv','E:/qrf/Lab Stuff/newDataset/facebook_labeled_sentences.csv',
                 'E:/qrf/Lab Stuff/newDataset/tensorflow_labeled_sentences.csv','E:/qrf/Lab Stuff/newDataset/vscode_labeled_sentences.csv',
                 'E:/qrf/Lab Stuff/newDataset/flutter_labeled_sentences.csv']
q_locations = ['E:/qrf/Lab Stuff/newDataset/spring-boot_PRs.txt','E:/qrf/Lab Stuff/newDataset/facebook_PRs.txt',
               'E:/qrf/Lab Stuff/newDataset/tensorflow_PRs.txt','E:/qrf/Lab Stuff/newDataset/vscode_PRs.txt',
               'E:/qrf/Lab Stuff/newDataset/flutter_PRs.txt']

In [3]:
def get_dataset(loc1,loc2):
    df = pd.read_csv(loc1)
    df2 = pd.read_table(loc2, sep = ',')
    issue_id = df.values[:,1].tolist()
    lbl_lst = df.values[:,3].tolist()
    all_issue = df2.values[:,1].tolist()
    question_lst = df2.values[:,11].tolist()
    title_lst = df2.values[:,7].tolist()

    newdic = dict(zip(issue_id,lbl_lst))
    pos_issue = list(set([i for i,j in newdic.items() if j == 0]))
    all_neg_issue = list(set([x for x in all_issue if x not in issue_id]))
    neg_issue = random.sample(all_neg_issue,len(pos_issue))
    pos_question = [q for t,q,i in zip(title_lst,question_lst,all_issue) if i in pos_issue]
    pos_title = [t for t,q,i in zip(title_lst,question_lst,all_issue) if i in pos_issue]

    neg_question = [q for t,q,i in zip(title_lst,question_lst,all_issue) if i in neg_issue]
    neg_title = [t for t,q,i in zip(title_lst,question_lst,all_issue) if i in neg_issue]

    title_lst = pos_title + neg_title
    question_lst= pos_question + neg_question
    lable_lst = [1 for i in range(len(pos_question))] + [0 for i in range(len(neg_question))]
    
    return title_lst,question_lst,lable_lst


In [4]:
title_list = []
question_list = []
lable_list = []
for loc1,loc2 in zip(pos_locations,q_locations):
    t1,q1,l1 = get_dataset(loc1,loc2)
    title_list +=t1
    question_list+=q1
    lable_list+=l1
    
print(len(title_list))
print(len(question_list))
print(len(lable_list))


29907
29907
29907


In [5]:
labeled_question = pd.DataFrame({'title':title_list,'question': question_list,'label':lable_list})

print(labeled_question.shape)
labeled_question.dropna(axis=0,how='any',inplace=True)
labeled_question['question'] = labeled_question['question'].str.lower()
labeled_question['title'] = labeled_question['title'].str.lower()
print(labeled_question.shape)
labeled_question.head()


(29907, 3)
(28998, 3)


,title,question,label
0,possible to use spring-boot on android develop...,<!-- thanks for raising a spring boot issue. p...,1
1,how do i use a custom websockethandler when i ...,my code is like this ```java @configuration @...,1
3,can i use a higher version of the `spring-boot...,can i use a higher version of the `spring-boot...,1
4,webflux functional endpoints authentication,i only found the documentation about authentic...,1
5,document that the minimum supported version of...,"hi, i deployed a project made with https://sta...",1


Data Cleaning

In [6]:
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from bs4 import BeautifulSoup
import re, string, unicodedata
from spellchecker import SpellChecker
import wordninja
import string
from nltk.corpus import wordnet
import concurrent.futures
from tqdm import tqdm

In [7]:
stop_words_en = list(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()

def expand_contractions(text):
    return contractions.fix(text)

spell = SpellChecker()

def correct_spelling(word):
    corrected_word = spell.correction(word)
    return str(corrected_word)

def split_words(word):
    split_word = wordninja.split(word)
    return ' '.join(split_word)

def cleanHTML_rmStopWords_lemmatize(df):
    df = df.apply(lambda x: expand_contractions(str(x)))
    df = df.apply(lambda x: str(x).lower())
    df = df.apply(lambda x: re.sub(re.compile('<.*?>'), '', str(x)))
    df = df.apply(lambda x: re.sub('\\b\\w[\\w.-]*@\\w+\\.\\w{2,4}\\b', 'email address', x))
    df = df.apply(lambda x: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'url address', x))
    df = df.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
    df = df.apply(lambda x: ' '.join(word for word in str(x).split() if word.lower() not in stop_words_en))
    #df = df.apply(lambda x: ' '.join(correct_spelling(word) for word in str(x).split()))
    df = df.apply(lambda x: ' '.join(split_words(word) for word in str(x).split()))
    df = df.apply(lambda x: re.sub(r'[^\w\s]+', '', x))
    df = df.apply(lambda x: ' '.join(lemmatizer.lemmatize(word) for word in str(x).split() if len(word)>2 and wordnet.synsets(word)))
    df = df.apply(lambda x: re.sub(' +', ' ', x))
    return df

cleaned_questions = cleanHTML_rmStopWords_lemmatize(labeled_question['question'])
cleaned_titles = cleanHTML_rmStopWords_lemmatize(labeled_question['title'])
cleaned_labeled_question = pd.DataFrame({'title':cleaned_titles,'body': cleaned_questions, 'label': labeled_question['label']})
cleaned_labeled_question = cleaned_labeled_question.loc[cleaned_labeled_question['body'].str.len()>10]

In [8]:
""" def html_remover(data):
  beauti = BeautifulSoup(data,'html.parser')
  return beauti.get_text()

# to remove URL
def url_remover(data):
  return re.sub(r'https\S','',data)

def web_associated(data):
  text = html_remover(data)
  text = url_remover(text)
  return text """

" def html_remover(data):\n  beauti = BeautifulSoup(data,'html.parser')\n  return beauti.get_text()\n\n# to remove URL\ndef url_remover(data):\n  return re.sub(r'https\\S','',data)\n\ndef web_associated(data):\n  text = html_remover(data)\n  text = url_remover(text)\n  return text "

In [9]:
""" def remove_round_brackets(data):
  return re.sub('\(.*?\)','',data)

def remove_punc(data):
  trans = str.maketrans('','', string.punctuation)
  return data.translate(trans)

def white_space(data):
  return ' '.join(data.split())

def complete_noise(data):
  new_data = remove_round_brackets(data)
  new_data = remove_punc(new_data)
  new_data = white_space(new_data)
  return new_data """

" def remove_round_brackets(data):\n  return re.sub('\\(.*?\\)','',data)\n\ndef remove_punc(data):\n  trans = str.maketrans('','', string.punctuation)\n  return data.translate(trans)\n\ndef white_space(data):\n  return ' '.join(data.split())\n\ndef complete_noise(data):\n  new_data = remove_round_brackets(data)\n  new_data = remove_punc(new_data)\n  new_data = white_space(new_data)\n  return new_data "

In [10]:
""" def Data_cleaning(x):
    return complete_noise(web_associated(x))

for index,row in labeled_question.iterrows():
    text = Data_cleaning(row['question'])
    #text1 = wordninja.split(text)
    #text2 = ' '.join(text1)
    labeled_question.loc[index,'question'] = text

cleaned_labeled_question = labeled_question.loc[labeled_question['question'].str.len()>10]
cleaned_labeled_question.head()
 """

" def Data_cleaning(x):\n    return complete_noise(web_associated(x))\n\nfor index,row in labeled_question.iterrows():\n    text = Data_cleaning(row['question'])\n    #text1 = wordninja.split(text)\n    #text2 = ' '.join(text1)\n    labeled_question.loc[index,'question'] = text\n\ncleaned_labeled_question = labeled_question.loc[labeled_question['question'].str.len()>10]\ncleaned_labeled_question.head()\n "

In [21]:
cleaned_labeled_question.head()

,title,body,label
1,use custom web socket handler configure web so...,code like java configuration enable web socket...,1
3,use higher version spring boot maven package l...,use higher version spring boot maven package l...,1
4,web flux functional endpoint authentication,found documentation authentication controller ...,1
5,document minimum supported version,deployed project made url address java followi...,1
6,anyway override filter bean registration assoc...,existing couple spring application extends spr...,1


In [22]:
cleaned_labeled_question.loc[1,'body']

'code like java configuration enable web socket message broker public class web socket implement web socket message broker configure override public void register stomp endpoint stomp endpoint registry stomp endpoint registry stomp endpoint registry add endpoint web socket set allowed origin pattern add interceptor web socket interceptor bean public web socket handler web socket handler return new web socket handler bean public handshake interceptor web socket interceptor return new web socket interceptor default sub protocol web socket handler used want use custom web socket handler configure take effect'

In [23]:
cleaned_labeled_question['label'].value_counts()

1    14545
0    14198
Name: label, dtype: int64

Output Data

In [28]:
cleaned_labeled_question.to_csv('labeled_t+b_for_all.csv', index=False, encoding='utf-8')

In [15]:
small_sample = cleaned_labeled_question.sample(3000)
small_sample.to_csv('labeled_t+b_small_sample.csv', index=False, encoding='utf-8')

In [29]:
mid_sample = cleaned_labeled_question.sample(10000)
mid_sample.to_csv('labeled_t+b_mid_sample.csv', index=False, encoding='utf-8')

In [30]:
mid_sample['label'].value_counts()

1    5047
0    4953
Name: label, dtype: int64

In [31]:
small_sample['label'].value_counts()

1    1551
0    1449
Name: label, dtype: int64

In [19]:
tl = mid_sample['title'].to_list()
ql = mid_sample['body'].to_list()
comlst = []
for i,j in zip(tl,ql):
    com = i + j
